In [ ]:
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom

def remove_background_slices(img_data, threshold=0, clip_min=-250):
    """
    Remove slices from all three dimensions where all voxels are <= threshold.
    
    Parameters:
    - img_data: 3D numpy array
    - threshold: background threshold value
    - clip_min: minimum value to clip voxels to (default: -250)
    
    Returns:
    - Cropped 3D numpy array
    """
    # Clip values below clip_min to clip_min
    img_data = np.clip(img_data, clip_min, None)
    
    # Find non-background slices for each dimension
    non_bg_slices = []
    
    for axis in range(3):
        # Check each slice along this axis
        max_vals = np.max(img_data, axis=tuple(i for i in range(3) if i != axis))
        non_bg_indices = np.where(max_vals > threshold)[0]
        
        if len(non_bg_indices) > 0:
            non_bg_slices.append((non_bg_indices[0], non_bg_indices[-1] + 1))
        else:
            # If all slices are background, keep at least one
            non_bg_slices.append((0, 1))
    
    # Crop the image
    cropped = img_data[
        non_bg_slices[0][0]:non_bg_slices[0][1],
        non_bg_slices[1][0]:non_bg_slices[1][1],
        non_bg_slices[2][0]:non_bg_slices[2][1]
    ]
    
    return cropped

def pad_or_crop_to_cube(img_data, target_size, pad_value=0):
    """
    Pad or crop image to cubic dimensions (target_size, target_size, target_size).
    When cropping: takes from start (0:target_size)
    When padding: centers the image (pads on both sides)
    
    Parameters:
    - img_data: 3D numpy array
    - target_size: desired cubic dimension
    - pad_value: value to use for padding (default: 0)
    
    Returns:
    - Cubic 3D numpy array
    """
    current_shape = img_data.shape
    result = np.full((target_size, target_size, target_size), pad_value, dtype=img_data.dtype)
    
    # Calculate start and end indices for each dimension
    for axis in range(3):
        current_dim = current_shape[axis]
        
        if current_dim >= target_size:
            # Crop: take from start (0:target_size)
            start_src = 0
            end_src = target_size
            start_dst = 0
            end_dst = target_size
        else:
            # Pad: place in center (pad on both sides)
            start_src = 0
            end_src = current_dim
            start_dst = (target_size - current_dim) // 2
            end_dst = start_dst + current_dim
        
        if axis == 0:
            x_src, x_dst = (start_src, end_src), (start_dst, end_dst)
        elif axis == 1:
            y_src, y_dst = (start_src, end_src), (start_dst, end_dst)
        else:
            z_src, z_dst = (start_src, end_src), (start_dst, end_dst)
    
    # Copy data
    result[x_dst[0]:x_dst[1], y_dst[0]:y_dst[1], z_dst[0]:z_dst[1]] = \
        img_data[x_src[0]:x_src[1], y_src[0]:y_src[1], z_src[0]:z_src[1]]
    
    return result

def resize_image(img_data, target_size):
    """
    Resize image to (target_size, target_size, target_size) using interpolation.
    
    Parameters:
    - img_data: 3D numpy array
    - target_size: desired output dimension
    
    Returns:
    - Resized 3D numpy array
    """
    current_shape = img_data.shape
    zoom_factors = [target_size / current_shape[i] for i in range(3)]
    
    # Use order=1 for linear interpolation (good for medical images)
    # Use order=0 for nearest neighbor (preserves label values for segmentation masks)
    resized = zoom(img_data, zoom_factors, order=1)
    
    return resized

def process_nifti(input_path, output_path, threshold=0, n=128, m=64, pad_value=0, clip_min=-250):
    """
    Complete pipeline to process NIfTI image.
    
    Parameters:
    - input_path: path to input .nii or .nii.gz file
    - output_path: path to save processed .nii.gz file
    - threshold: background threshold for slice removal
    - n: cubic dimension after cropping/padding
    - m: final dimension after resizing
    - pad_value: value to use for padding (default: 0)
    - clip_min: minimum value to clip voxels to (default: -250)
    """
    # Load the NIfTI image
    # print(f"Loading {input_path}...")
    nii_img = nib.load(input_path)
    img_data = nii_img.get_fdata()
    
    # print(f"Original shape: {img_data.shape}")
    
    # Step 1: Remove background slices
    # print(f"Removing background slices (threshold={threshold})...")
    cropped = remove_background_slices(img_data, threshold, clip_min)
    # print(f"After cropping: {cropped.shape}")
    
    # Step 2: Pad/crop to cubic dimensions
    # print(f"Padding/cropping to ({n}, {n}, {n})...")
    cubic = pad_or_crop_to_cube(cropped, n, pad_value)
    # print(f"After cubic transform: {cubic.shape}")
    
    # Step 3: Resize to final dimensions
    # print(f"Resizing to ({m}, {m}, {m})...")
    resized = resize_image(cubic, m)
    # print(f"Final shape: {resized.shape}")
    
    # Create new NIfTI image with updated affine matrix
    # Scale the affine to account for the resize
    new_affine = nii_img.affine.copy()
    scale_factor = img_data.shape[0] / m  # Approximate scaling
    new_affine[:3, :3] = new_affine[:3, :3] * scale_factor
    
    # Save the processed image
    # print(f"Saving to {output_path}...")
    new_nii = nib.Nifti1Image(resized, new_affine)
    nib.save(new_nii, output_path)
    # print("Done!")
    
    return resized

# # Example usage
# if __name__ == "__main__":
#     # Customize these parameters
#     input_file = r"d:\Kananat\Data\2_Masked\47-4881 L 2014_masked.nii.gz"
#     output_file = "output_image.nii.gz"
#     threshold = -250  # Background threshold
#     n = 128  # Cubic dimension
#     m = 128   # Final dimension
#     pad_value = -250  # Value to use for padding
#     clip_min = -250  # Minimum voxel value (values below this will be set to this)
    
#     processed_img = process_nifti(
#         input_path=input_file,
#         output_path=output_file,
#         threshold=threshold,
#         n=n,
#         m=m,
#         pad_value=pad_value,
#         clip_min=clip_min
#     )

In [13]:
from pathlib import Path

input_folder = Path(r"D:\Kananat\Data\processing_More_data\preprocessed\Masked")
output_folder = r"D:\Kananat\Data\processing_More_data\to_add_cropped"

i = 0
for nii_file in input_folder.glob("*.nii*"):
    i += 1
    print(f"Processing file {i}: {nii_file.name}")
    output_path = Path(output_folder) / nii_file.name
    processed_img = process_nifti(
        input_path=str(nii_file),
        output_path=str(output_path),
        threshold=-250,
        n=500,
        m=256,
        pad_value=-250,
        clip_min=-250
    )

Processing file 1: 5011620_2025_02_13_L_masked.nii.gz
Processing file 2: 5011620_2025_02_13_R_masked.nii.gz
Processing file 3: 5218852_2016_05_26_L_masked.nii.gz
Processing file 4: 5218852_2016_05_26_R_masked.nii.gz
Processing file 5: 5237193_2016_04_05_L_masked.nii.gz
Processing file 6: 5237193_2016_04_05_R_masked.nii.gz
Processing file 7: 5425927_2016_04_24_L_masked.nii.gz
Processing file 8: 5425927_2016_04_24_R_masked.nii.gz
Processing file 9: 5450_2016_02_26_L_masked.nii.gz
Processing file 10: 5450_2016_02_26_R_masked.nii.gz
Processing file 11: 5630067_2024_01_31_R_masked.nii.gz
Processing file 12: 5724983_2016_07_02_L_masked.nii.gz
Processing file 13: 5724983_2016_07_02_R_masked.nii.gz
Processing file 14: 5734770_2016_01_17_L_masked.nii.gz
Processing file 15: 5734770_2016_01_17_R_masked.nii.gz
Processing file 16: 5841636_2016_01_29_L_masked.nii.gz
Processing file 17: 5841636_2016_01_29_R_masked.nii.gz
Processing file 18: 592326_2016_02_20_L_masked.nii.gz
Processing file 19: 592326

In [18]:
input_folder = Path(r"D:\Kananat\Data\processing_More_data\to_add_cropped")

for file in input_folder.glob("*.nii.gz"):
    parts = file.name.split("_")
    new_name = f"{parts[0][:2]}-{parts[0][2:]} {parts[4]} {parts[1]}{parts[2]}{parts[3]}.nii.gz"
    new_path = file.parent / new_name
    print(file.name, new_name)
    file.rename(new_path)

5011620_2025_02_13_L_masked.nii.gz 50-11620 L 20250213.nii.gz
5011620_2025_02_13_R_masked.nii.gz 50-11620 R 20250213.nii.gz
5218852_2016_05_26_L_masked.nii.gz 52-18852 L 20160526.nii.gz
5218852_2016_05_26_R_masked.nii.gz 52-18852 R 20160526.nii.gz
5237193_2016_04_05_L_masked.nii.gz 52-37193 L 20160405.nii.gz
5237193_2016_04_05_R_masked.nii.gz 52-37193 R 20160405.nii.gz
5425927_2016_04_24_L_masked.nii.gz 54-25927 L 20160424.nii.gz
5425927_2016_04_24_R_masked.nii.gz 54-25927 R 20160424.nii.gz
5450_2016_02_26_L_masked.nii.gz 54-50 L 20160226.nii.gz
5450_2016_02_26_R_masked.nii.gz 54-50 R 20160226.nii.gz
5630067_2024_01_31_R_masked.nii.gz 56-30067 R 20240131.nii.gz
5724983_2016_07_02_L_masked.nii.gz 57-24983 L 20160702.nii.gz
5724983_2016_07_02_R_masked.nii.gz 57-24983 R 20160702.nii.gz
5734770_2016_01_17_L_masked.nii.gz 57-34770 L 20160117.nii.gz
5734770_2016_01_17_R_masked.nii.gz 57-34770 R 20160117.nii.gz
5841636_2016_01_29_L_masked.nii.gz 58-41636 L 20160129.nii.gz
5841636_2016_01_29_R